# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
! python -m pip install pandas

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,-6.36,85,100,3.75,CA,1680112856
1,1,makakilo city,21.3469,-158.0858,25.18,78,20,6.69,US,1680113247
2,2,phan thiet,10.9333,108.1000,23.96,89,97,2.80,VN,1680113455
3,3,hobart,-42.8794,147.3294,12.92,79,100,5.14,AU,1680112954
4,4,ushuaia,-54.8000,-68.3000,4.34,66,81,2.59,AR,1680112936


In [16]:
city_data_df = city_data_df.rename(columns={'Lat': 'latitude', 'Lng': 'longitude'})
city_data_df.head()

,City_ID,City,latitude,longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,thompson,55.7435,-97.8558,-6.36,85,100,3.75,CA,1680112856
1,1,makakilo city,21.3469,-158.0858,25.18,78,20,6.69,US,1680113247
2,2,phan thiet,10.9333,108.1000,23.96,89,97,2.80,VN,1680113455
3,3,hobart,-42.8794,147.3294,12.92,79,100,5.14,AU,1680112954
4,4,ushuaia,-54.8000,-68.3000,4.34,66,81,2.59,AR,1680112936


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
                                    "longitude"
                                     ,"latitude"
                                     , geo=True
                                     , tiles="EsriImagery"
                                     , frame_width=600
                                     , frame_height=500
                                     , size = 'Humidity'
                                     , scale = 0.5
                                     , color = 'City'
                                     , xlabel='Longitude'
                                     , ylabel='Latitude'
                    )
# Display the map
map_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21), :] 
city_data_df2 = city_data_df2.loc[(city_data_df2['Wind Speed'] < 4.5) | (city_data_df2['Cloudiness'] == 0), :]
# city_data_df2.describe()
# Drop any rows with null values
city_data_df3 = city_data_df2.dropna()
# Display sample data
city_data_df3.head()

,City_ID,City,latitude,longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,phan thiet,10.9333,108.1000,23.96,89,97,2.80,VN,1680113455
10,10,honiara,-9.4333,159.9500,25.27,89,100,1.92,SB,1680113457
14,14,hilo,19.7297,-155.0900,24.04,93,75,3.09,US,1680113079
20,20,carnarvon,-24.8667,113.6333,24.31,75,100,2.76,AU,1680113100
33,33,moctezuma,29.8000,-109.6667,25.80,11,0,4.21,MX,1680113462


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# new_df = pd.DataFrame({'A': df['A'], 'B': df['B']})
hotel_df = pd.DataFrame({'City': city_data_df3["City"], 'Country': city_data_df3["Country"], 'latitude': city_data_df3["latitude"],'longitude': city_data_df3["longitude"],'Humidity': city_data_df3["Humidity"]})
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,latitude,longitude,Humidity,Hotel Name
2,phan thiet,VN,10.9333,108.1000,89,
10,honiara,SB,-9.4333,159.9500,89,
14,hilo,US,19.7297,-155.0900,93,
20,carnarvon,AU,-24.8667,113.6333,75,
33,moctezuma,MX,29.8000,-109.6667,11,
...,...,...,...,...,...,...
539,mount isa,AU,-20.7333,139.5000,57,
541,savannah bight,HN,16.4500,-85.8500,70,
548,maxixe,MZ,-23.8597,35.3472,77,
559,nabire,ID,-3.3667,135.4833,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
#     "lang" : "en",
    "apiKey": geoapify_key,
    "limit":5
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
     # Get the city's name
    city = row["City"]
    # Get latitude, longitude from the DataFrame
    latitude = row["latitude"]
    longitude = row["longitude"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
phan thiet - nearest hotel: Khách sạn Cà Ty
honiara - nearest hotel: King Solomon Hotel
hilo - nearest hotel: Dolphin Bay Hotel
carnarvon - nearest hotel: No hotel found
moctezuma - nearest hotel: Hotel San Antonio
riyadh - nearest hotel: White Palace Hotel
san nicolas - nearest hotel: Mira de Polaris Hotel
wewak - nearest hotel: Village Inn
kokopo - nearest hotel: Hotel Kokopo
avarua - nearest hotel: Paradise Inn
lorengau - nearest hotel: Seeadler Bay Hotel
olinda - nearest hotel: Fidji Motel
marzuq - nearest hotel: No hotel found
castro - nearest hotel: Hotel Central Palace
kendari - nearest hotel: City Hotel Kendari
rikitea - nearest hotel: Chez Bianca & Benoit
baikunthpur - nearest hotel: No hotel found
tokonou - nearest hotel: No hotel found
safaga - nearest hotel: Dolphin Hotel
benguela - nearest hotel: Hotel Moibela
vaini - nearest hotel: Keleti International Resort
sao filipe - nearest hotel: Bela Vista
tiangua - nearest hotel: Pousada
saint-philippe - nea

,City,Country,latitude,longitude,Humidity,Hotel Name
2,phan thiet,VN,10.9333,108.1000,89,Khách sạn Cà Ty
10,honiara,SB,-9.4333,159.9500,89,King Solomon Hotel
14,hilo,US,19.7297,-155.0900,93,Dolphin Bay Hotel
20,carnarvon,AU,-24.8667,113.6333,75,No hotel found
33,moctezuma,MX,29.8000,-109.6667,11,Hotel San Antonio
...,...,...,...,...,...,...
539,mount isa,AU,-20.7333,139.5000,57,Ibis Styles
541,savannah bight,HN,16.4500,-85.8500,70,La Giralda
548,maxixe,MZ,-23.8597,35.3472,77,Hotel Herry
559,nabire,ID,-3.3667,135.4833,91,Hotel Anggrek


In [26]:
hotel_df.sample(20)

,City,Country,latitude,longitude,Humidity,Hotel Name
158,bengkulu,ID,-3.8004,102.2655,87,Dena Hotel
472,grand gaube,MU,-20.0064,57.6608,75,Coco Villa
320,buala,SB,-8.1450,159.5921,84,No hotel found
39,kokopo,PG,-4.3520,152.2633,85,Hotel Kokopo
518,kahului,US,20.8947,-156.4700,81,Maui Seaside Hotel
332,mayumba,GA,-3.4320,10.6554,83,Résidence Hôtelière Fabso
245,port-gentil,GA,-0.7193,8.7815,70,Tara-Mé
14,hilo,US,19.7297,-155.0900,93,Dolphin Bay Hotel
210,namatanai,PG,-3.6667,152.4333,85,Namatanai Hotel
457,nchelenge,ZM,-9.3451,28.7340,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display
# Configure the map plot
map_2 = hotel_df.hvplot.points(
                                    "longitude"
                                     ,"latitude"
                                     , geo=True
                                     , tiles="EsriImagery"
                                     , frame_width=600
                                     , frame_height=500
                                     , size = 'Humidity'
                                     , scale = 0.5
                                     , color = 'City'
                                     , hover_cols=['Hotel Name', 'Country']
                                     , xlabel='Longitude'
                                     , ylabel='Latitude'
                    )
# Display the map
map_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,Hotel Name,Country)